This script creates Multidirectional, Oblique-weighted, shaded-relief image from a DEM

Somewhat replicates the method described by Robert Mark:

    Multidirectional, oblique-weighted, shaded-relief image of the Island of Hawaii

    USGS OF-92-422 (not dated)

    https://pubs.usgs.gov/of/1992/of92-422/of92-422.pdf

Uses arcpy methods but could easily be adapted to rasterio or gdal methods


In [1]:
#Import packages
import arcpy
from arcpy import env
from arcpy.sa import *
import math
import os

In [2]:
def hshade(dem):
     '''
     Creates hillshades a different azimuths, directions defined in list azimuths
     Saves each hillshade as shade{dg}.tif
     '''
     azimuths = [225, 270, 315, 360]
     for dg in azimuths:
          print(f"Calculating hillshade: {dg}")
          shade = Hillshade(dem, dg)
          outshd = "shade" + str(dg) + ".tif"
          shade.save(outshd)

In [3]:
def wraster(dem, ncells = 3):
     '''
     Calculates weight rasters that will multiply each corresponding shaded relief.
     In the original paper, Robert Mark resamples the DEM to a lower resolution 
       and blurs the DEM due to processing limitations at the time.
       I commented out the part corresponding to the blurring.
     '''
     dg2rad = math.pi/180
     azimuths = [225, 270, 315, 360]
     #weights = []
     #print("Blurring the dem...")
     #h01 = FocalStatistics(dem, NbrCircle(ncells, "CELL"))
     #h02 = FocalStatistics(Raster(h01), NbrCircle(ncells, "CELL"))
     #h03 = FocalStatistics(Raster(h02), NbrCircle(ncells, "CELL"))
     #asp = Aspect(Raster(h03))
     asp = Aspect(dem)
     #asp1 = Con(IsNull(Raster(asp)), 293, asp)
     outasp = "asp1.tif"
     asp.save(outasp)
     for dg in azimuths:
          print(f"Calculating weight raster: {dg}")
          #What does this equation do? It gives more weight to slopes
          # that are perpendicular to the source of light. Aspects
          # either facing the source of light or facing opposite the source of
          # light will receive small weights.
          w = Square(Sin((Raster(outasp) - dg) * dg2rad))
          wout = "w" + str(dg) + ".tif"
          w.save(wout)

In [4]:
def mdwshd():
     #Collect weight rasters created by the function above into a list
     wlist = [f for f in os.listdir() if f.startswith("w") and f.endswith("tif")]
     print(wlist)
     #Collect the shaded relief rasters created above into a list
     shdlist = [f for f in os.listdir() if f.startswith("shade") and f.endswith("tif")]
     print(shdlist)
     tmp0 = (Raster(wlist[0])*Raster(shdlist[0]))
     tmp1 = (Raster(wlist[1])*Raster(shdlist[1]))
     tmp2 = (Raster(wlist[2])*Raster(shdlist[2])) 
     tmp3 = (Raster(wlist[3])*Raster(shdlist[3])) 
     tmp = Raster(tmp0) + Raster(tmp1) + Raster(tmp2) + Raster(tmp3)
     #In the original paper, RM divides by 2 
     # You can divide by 4 to get the average. If the color ramp assigns colors based on the stretch [min, max] values
     #  this shouldn't matter.
     shd = Int(Raster(tmp) / 2)
     #Multiple direction hillshade saved as:
     outshd = "mdwshd.tif"
     shd.save(outshd)
     print(f"Multidirectional, weighted, shaded-relief raster calculated.\nSaved as '{outshd}'")

In [7]:
def main():
     #Set the path to where your dem is
     path = r'H:\\GRG470C\\MDWH'
     env.workspace = path
     os.chdir(path)
     #Overwrite file option set to True
     env.overwriteOutput = True
     #Pass on the name of the DEM to the variable elv
     elv = "dem_roi.tif"
     hshade(elv)
     wraster(elv)
     mdwshd()

In [ ]:
main()